# 4 assets

In [69]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize #the optomizaton lib we ues

covariance_csv_path = r'F:\Learning_journal_at_CUHK\FTEC5610_Computational_Finance\Assignment\Assignment 1\Part1_calculation\Annualized covariance between ETF.csv'
covariance_df = pd.read_csv(covariance_csv_path)
num_assets = 4
cov_matrix = covariance_df.values[:num_assets, :num_assets]
asset_names = covariance_df.columns.tolist()[:num_assets]

In [70]:
covariance_df

,2823,3199,2840,3175,3046
0,0.08137,-0.00027,0.00546,0.01881,0.03654
1,-0.00027,0.00279,0.00077,0.00078,0.00002
2,0.00546,0.00077,0.03070,0.01293,-0.00571
3,0.01881,0.00078,0.01293,0.11040,0.05456
4,0.03654,0.00002,-0.00571,0.05456,0.63990


In [71]:
cov_matrix

array([[ 0.08137, -0.00027,  0.00546,  0.01881],
       [-0.00027,  0.00279,  0.00077,  0.00078],
       [ 0.00546,  0.00077,  0.0307 ,  0.01293],
       [ 0.01881,  0.00078,  0.01293,  0.1104 ]])

In [72]:
asset_names

['2823', '3199', '2840', '3175']

In [73]:
def erc_objective_function(weights, cov_matrix):
    # constarint into the objective function which can accelerate processing, although we set in min function
    # standardize weights
    weights = np.array(weights) / np.sum(weights)
    # calculate covariance contributions: ω * (Ωω)
    # cov_contrib is a vector of length n_assets
    cov_contrib = weights * (cov_matrix @ weights)
    # sum of squared differences between risk contributions
    # use auto broadcasting to calculate the pairwise differences
    # it calculate all the differences between the elements in cov_contrib 
    # use sum to get a scalar, although it is a digonal matrix, which is 2 times of what we want, but it does not matter
    return np.sum((cov_contrib[:, None] - cov_contrib[None, :])**2)


def analyze_risk_contributions(weights, cov_matrix, asset_names):
    # calculate RC
    # to verify validity of the optimizer
    # standardize weights
    weights = weights / np.sum(weights)
    # calculate sigma_p
    portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    # calculate MR
    mrc = (cov_matrix @ weights) / portfolio_vol
    # calculate RC
    trc = weights * mrc
    # percentage
    crc = trc / np.sum(trc)
    return pd.DataFrame({
        'Weight': weights,
        'Marginal_Risk_Contribution': mrc ,
        'Risk_Contribution': crc, 
    }, index=asset_names)

In [74]:
# select and compare different optimizers
# config setup using average weight as initial guess, we will also compare different initial guessing methods later
initial_weights = np.array([1/num_assets] * num_assets)
# to store optimizer results
optimizer_results = []
# try for every optimizer
for method in ['SLSQP', 'trust-constr', 'COBYLA']:
    # constraint 1 : weight sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    # constraint 2: weights >= 0, which means weights between 0 and 1 considering constraint 1
    bounds = tuple((0, 1) for _ in range(num_assets))

    # special handling for COBYLA
    if method == 'COBYLA': 
        # convert constraint 1 and 2 to inequalities to fit COBYLA
        constraints = [{'type': 'ineq', 'fun': lambda w: w[i]} for i in range(num_assets)]# w_i >= 0
        constraints.append({'type': 'ineq', 'fun': lambda w: 1 - np.sum(w)})#ineq means >=0
        constraints.append({'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})
        bounds = None
        
    # ininitialize and run optimizer
    result = minimize(erc_objective_function, initial_weights, args=(cov_matrix,), method=method,
                      bounds=bounds, constraints=constraints, options={'maxiter': 2000})
    
    # to get final weights, weights are final outcome and is neccessarily sum to 1
    final_weights = result.x / np.sum(result.x)

    portfolio_variance = final_weights.T @ cov_matrix @ final_weights
    portfolio_volatility = np.sqrt(portfolio_variance)

    # calculate RC
    analysis = analyze_risk_contributions(final_weights, cov_matrix, asset_names)

    # store results
    optimizer_results.append({
        'Method': method, 
        'Success': result.success, 
        'Iterations': result.nfev,
        'Final Objective': np.round(result.fun, 5),
        'Portfolio Variance': np.round(portfolio_variance, 5),
        'Portfolio Volatility': np.round(portfolio_volatility, 5),
        'Final Weights': list(np.round(final_weights, 5)),
        'Risk Contrib': list(np.round(analysis['Risk_Contribution'], 5)),
        'Marginal Risk Contrib': list(np.round(analysis['Marginal_Risk_Contribution'], 5)) 
    })


df_optimizers = pd.DataFrame(optimizer_results).set_index('Method')

c:\Users\10158\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:504: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


In [75]:
df_optimizers

,Success,Iterations,Final Objective,Portfolio Variance,Portfolio Volatility,Final Weights,Risk Contrib,Marginal Risk Contrib
Method,,,,,,,,
SLSQP,True,61,0.00006,0.01057,0.10279,"[0.15393, 0.37533, 0.33504, 0.1357]","[0.24482, 0.04864, 0.41758, 0.28896]","[0.16349, 0.01332, 0.12812, 0.21889]"
trust-constr,False,9980,0.00034,0.01882,0.13719,"[0.24989, 0.25057, 0.25063, 0.2489]","[0.3494, 0.01356, 0.16605, 0.47098]","[0.19182, 0.00743, 0.0909, 0.2596]"
COBYLA,True,98,0.00000,0.00494,0.07031,"[0.10947, 0.63599, 0.16796, 0.08658]","[0.24983, 0.2498, 0.25016, 0.25021]","[0.16046, 0.02762, 0.10472, 0.20318]"


In [76]:
df_optimizers.to_csv('ERC_4_assets_optimizers_comparison.csv', index=True)

Based on the comparative analysis of three optimizers for the Equal Risk Contribution (ERC) portfolio, **COBYLA is demonstrably the optimal choice.**

It was the only method to successfully converge to a near-perfect solution, achieving almost equal risk contributions of approximately 25% from each asset and a final objective value of virtually zero (~10⁻¹¹). In contrast, both `SLSQP` and `trust-constr` failed to produce a valid ERC portfolio. Notably, the `Success: True` status from `SLSQP` was misleading, as its resulting risk distribution was highly unequal. Therefore, COBYLA is the most effective and reliable optimizer for this specific problem configuration.

By the way, although `COBYLA` iterations is kindly more than `SLSQP`, it is acceptable, regardless `SLSQP` is not so equal risk contributing.

In [77]:
# compare and initial guessing methods using the best optimizer (COBYLA)
# different initializations
initializations = {
    'Equal Weights': np.array([1/num_assets] * num_assets),
    'Inverse Volatility': (1 / np.diag(cov_matrix)) / np.sum(1 / np.diag(cov_matrix)),
    'Random Weights': np.random.dirichlet(np.ones(num_assets), size=1)[0],
    'First Asset Only': np.array([1.0] + [0.0] * (num_assets - 1))
}
# to store results
initialization_results = []
# run for each initialization
for name, init_weights in initializations.items():
    # constraints for COBYLA
    constraints = [{'type': 'ineq', 'fun': lambda w: w[i]} for i in range(num_assets)] # w_i >= 0
    constraints.append({'type': 'ineq', 'fun': lambda w: 1 - np.sum(w)})             # sum(w) <= 1
    constraints.append({'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})             # sum(w) >= 1
    bounds = None
    # ininitialize and run optimizer
    result = minimize(erc_objective_function, init_weights, args=(cov_matrix,), method='COBYLA',
                      bounds=bounds, constraints=constraints, options={'maxiter': 2000})
    # to get final weights, weights are final outcome and is neccessarily sum to 1
    final_weights = result.x / np.sum(result.x)
    # calculate RC
    analysis = analyze_risk_contributions(final_weights, cov_matrix, asset_names)

    initialization_results.append({
        'Initialization': name, 
        'Success': result.success, 
        'Iterations': result.nfev,
        'Final Objective': np.round(result.fun, 5),
        'Portfolio Variance': np.round(portfolio_variance, 5),
        'Portfolio Volatility': np.round(portfolio_volatility, 5),
        'Final Weights': list(np.round(final_weights, 5)),
        'Risk Contrib': list(np.round(analysis['Risk_Contribution'], 5)),
        'Marginal Risk Contrib': list(np.round(analysis['Marginal_Risk_Contribution'], 5)) 
    })

df_initializations_cobyla = pd.DataFrame(initialization_results).set_index('Initialization')

In [78]:
df_initializations_cobyla

,Success,Iterations,Final Objective,Portfolio Variance,Portfolio Volatility,Final Weights,Risk Contrib,Marginal Risk Contrib
Initialization,,,,,,,,
Equal Weights,True,98,0.0,0.00494,0.07031,"[0.10947, 0.63599, 0.16796, 0.08658]","[0.24983, 0.2498, 0.25016, 0.25021]","[0.16046, 0.02762, 0.10472, 0.20318]"
Inverse Volatility,True,92,0.0,0.00494,0.07031,"[0.10937, 0.63646, 0.16768, 0.0865]","[0.24969, 0.25049, 0.24977, 0.25005]","[0.1604, 0.02765, 0.10466, 0.20312]"
Random Weights,True,74,0.0,0.00494,0.07031,"[-0.18461, 0.80814, 0.23977, 0.1367]","[0.24997, 0.25004, 0.24987, 0.25012]","[-0.12402, 0.02834, 0.09545, 0.16759]"
First Asset Only,True,123,0.0,0.00494,0.07031,"[0.17901, 1.02847, -0.3677, 0.16022]","[0.24991, 0.24999, 0.24996, 0.25013]","[0.14612, 0.02544, -0.07115, 0.1634]"


In [79]:
df_initializations_cobyla.to_csv('ERC_4_assets_initializations_comparison_cobyla.csv', index=True)

This analysis confirms that `COBYLA` is a highly effective optimizer for achieving the Equal Risk Contribution (ERC) objective, successfully minimizing the function from all four initial points. A critical finding, however, is that `COBYLA` violated the non-negativity constraints for the `Random Weights` and `First Asset Only` initializations, producing invalid portfolios that included short positions, which due to the unavoid inherent of this method.

Conversely, both the `Equal Weights` and `Inverse Volatility` methods yielded valid and consistent outcomes, converging to the true optimal ERC solution.

Therefore, the portfolio derived from the **`Equal Weights` initialization is selected as the final result.** This method is preferred not only because it is a common and stable industry practice, but also because it reliably produced a valid and optimal portfolio in this analysis.

# 5 assets

In [80]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize #the optomizaton lib we ues

covariance_csv_path = r'F:\Learning_journal_at_CUHK\FTEC5610_Computational_Finance\Assignment\Assignment 1\Part1_calculation\Annualized covariance between ETF.csv'
covariance_df = pd.read_csv(covariance_csv_path)
num_assets = 5
cov_matrix = covariance_df.values[:num_assets, :num_assets]
asset_names = covariance_df.columns.tolist()[:num_assets]

In [81]:
covariance_df

,2823,3199,2840,3175,3046
0,0.08137,-0.00027,0.00546,0.01881,0.03654
1,-0.00027,0.00279,0.00077,0.00078,0.00002
2,0.00546,0.00077,0.03070,0.01293,-0.00571
3,0.01881,0.00078,0.01293,0.11040,0.05456
4,0.03654,0.00002,-0.00571,0.05456,0.63990


In [82]:
asset_names

['2823', '3199', '2840', '3175', '3046']

In [83]:
def erc_objective_function(weights, cov_matrix):
    # constarint into the objective function which can accelerate processing, although we set in min function
    # standardize weights
    weights = np.array(weights) / np.sum(weights)
    # calculate covariance contributions: ω * (Ωω)
    # cov_contrib is a vector of length n_assets
    cov_contrib = weights * (cov_matrix @ weights)
    # sum of squared differences between risk contributions
    # use auto broadcasting to calculate the pairwise differences
    # it calculate all the differences between the elements in cov_contrib 
    # use sum to get a scalar, although it is a digonal matrix, which is 2 times of what we want, but it does not matter
    return np.sum((cov_contrib[:, None] - cov_contrib[None, :])**2)


def analyze_risk_contributions(weights, cov_matrix, asset_names):
    # calculate RC
    # to verify validity of the optimizer
    # standardize weights
    weights = weights / np.sum(weights)
    # calculate sigma_p
    portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    # calculate MR
    mrc = (cov_matrix @ weights) / portfolio_vol
    # calculate RC
    trc = weights * mrc
    # percentage
    crc = trc / np.sum(trc)
    return pd.DataFrame({
        'Weight': weights,
        'Marginal_Risk_Contribution': mrc ,
        'Risk_Contribution': crc, 
    }, index=asset_names)

In [84]:
# select and compare different optimizers
# config setup using average weight as initial guess, we will also compare different initial guessing methods later
initial_weights = np.array([1/num_assets] * num_assets)
# to store optimizer results
optimizer_results = []
# try for every optimizer
for method in ['SLSQP', 'trust-constr', 'COBYLA']:
    # constraint 1 : weight sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    # constraint 2: weights >= 0, which means weights between 0 and 1 considering constraint 1
    bounds = tuple((0, 1) for _ in range(num_assets))

    # special handling for COBYLA
    if method == 'COBYLA': 
        # convert constraint 1 and 2 to inequalities to fit COBYLA
        constraints = [{'type': 'ineq', 'fun': lambda w: w[i]} for i in range(num_assets)]# w_i >= 0
        constraints.append({'type': 'ineq', 'fun': lambda w: 1 - np.sum(w)})#ineq means >=0
        constraints.append({'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})
        bounds = None
        
    # ininitialize and run optimizer
    result = minimize(erc_objective_function, initial_weights, args=(cov_matrix,), method=method,
                      bounds=bounds, constraints=constraints, options={'maxiter': 2000})
    
    # to get final weights, weights are final outcome and is neccessarily sum to 1
    final_weights = result.x / np.sum(result.x)

    portfolio_variance = final_weights.T @ cov_matrix @ final_weights
    portfolio_volatility = np.sqrt(portfolio_variance)

    # calculate RC
    analysis = analyze_risk_contributions(final_weights, cov_matrix, asset_names)

    # store results
    optimizer_results.append({
        'Method': method, 
        'Success': result.success, 
        'Iterations': result.nfev,
        'Final Objective': np.round(result.fun, 5),
        'Portfolio Variance': np.round(portfolio_variance, 5),
        'Portfolio Volatility': np.round(portfolio_volatility, 5),
        'Final Weights': list(np.round(final_weights, 5)),
        'Risk Contrib': list(np.round(analysis['Risk_Contribution'], 5)),
        'Marginal Risk Contrib': list(np.round(analysis['Marginal_Risk_Contribution'], 5)) 
    })


df_optimizers = pd.DataFrame(optimizer_results).set_index('Method')

c:\Users\10158\anaconda3\Lib\site-packages\scipy\optimize\_differentiable_functions.py:504: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


In [85]:
df_optimizers

,Success,Iterations,Final Objective,Portfolio Variance,Portfolio Volatility,Final Weights,Risk Contrib,Marginal Risk Contrib
Method,,,,,,,,
SLSQP,True,126,0.00005,0.01119,0.10579,"[0.12027, 0.3931, 0.3135, 0.11925, 0.05388]","[0.16769, 0.04917, 0.33105, 0.24218, 0.20991]","[0.1475, 0.01323, 0.11171, 0.21484, 0.41212]"
trust-constr,False,11982,0.00123,0.03079,0.17548,"[0.21581, 0.21655, 0.21962, 0.21143, 0.13659]","[0.19391, 0.00621, 0.07161, 0.25996, 0.46831]","[0.15768, 0.00503, 0.05722, 0.21576, 0.60164]"
COBYLA,True,230,0.00000,0.00580,0.07618,"[0.09937, 0.6197, 0.16628, 0.07704, 0.03762]","[0.19949, 0.20194, 0.19788, 0.19993, 0.20077]","[0.15293, 0.02482, 0.09065, 0.1977, 0.40654]"


In [86]:
df_optimizers.to_csv('ERC_5_assets_optimizers_comparison.csv', index=True)

Likely, `COBYLA` is the best optimizer.

In [87]:
# compare and initial guessing methods using the best optimizer (COBYLA)
# different initializations
initializations = {
    'Equal Weights': np.array([1/num_assets] * num_assets),
    'Inverse Volatility': (1 / np.diag(cov_matrix)) / np.sum(1 / np.diag(cov_matrix)),
    'Random Weights': np.random.dirichlet(np.ones(num_assets), size=1)[0],
    'First Asset Only': np.array([1.0] + [0.0] * (num_assets - 1))
}
# to store results
initialization_results = []
# run for each initialization
for name, init_weights in initializations.items():
    # constraints for COBYLA
    constraints = [{'type': 'ineq', 'fun': lambda w: w[i]} for i in range(num_assets)] # w_i >= 0
    constraints.append({'type': 'ineq', 'fun': lambda w: 1 - np.sum(w)})             # sum(w) <= 1
    constraints.append({'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})             # sum(w) >= 1
    bounds = None
    # ininitialize and run optimizer
    result = minimize(erc_objective_function, init_weights, args=(cov_matrix,), method='COBYLA',
                      bounds=bounds, constraints=constraints, options={'maxiter': 2000})
    # to get final weights, weights are final outcome and is neccessarily sum to 1
    final_weights = result.x / np.sum(result.x)
    # calculate RC
    analysis = analyze_risk_contributions(final_weights, cov_matrix, asset_names)

    initialization_results.append({
        'Initialization': name, 
        'Success': result.success, 
        'Iterations': result.nfev,
        'Final Objective': np.round(result.fun, 5),
        'Portfolio Variance': np.round(portfolio_variance, 5),
        'Portfolio Volatility': np.round(portfolio_volatility, 5),
        'Final Weights': list(np.round(final_weights, 5)),
        'Risk Contrib': list(np.round(analysis['Risk_Contribution'], 5)),
        'Marginal Risk Contrib': list(np.round(analysis['Marginal_Risk_Contribution'], 5)) 
    })

df_initializations_cobyla = pd.DataFrame(initialization_results).set_index('Initialization')

In [88]:
df_initializations_cobyla

,Success,Iterations,Final Objective,Portfolio Variance,Portfolio Volatility,Final Weights,Risk Contrib,Marginal Risk Contrib
Initialization,,,,,,,,
Equal Weights,True,230,0.0,0.0058,0.07618,"[0.09937, 0.6197, 0.16628, 0.07704, 0.03762]","[0.19949, 0.20194, 0.19788, 0.19993, 0.20077]","[0.15293, 0.02482, 0.09065, 0.1977, 0.40654]"
Inverse Volatility,True,218,0.0,0.0058,0.07618,"[0.09956, 0.61883, 0.16708, 0.07693, 0.0376]","[0.19989, 0.20117, 0.19923, 0.19939, 0.20032]","[0.15305, 0.02478, 0.0909, 0.19757, 0.40609]"
Random Weights,True,146,0.0,0.0058,0.07618,"[0.13667, 0.73568, 0.24028, -0.16546, 0.05283]","[0.2001, 0.20199, 0.19887, 0.20017, 0.19886]","[0.12721, 0.02386, 0.07191, -0.10511, 0.32705]"
First Asset Only,True,219,0.0,0.0058,0.07618,"[0.16181, 0.98711, -0.3419, 0.14134, 0.05164]","[0.20041, 0.20078, 0.19897, 0.20017, 0.19967]","[0.13891, 0.02281, -0.06527, 0.15883, 0.43368]"


Likely, we choose `Equal Weights` as initialization.

In [89]:
df_initializations_cobyla.to_csv('ERC_5_assets_initializations_comparison_cobyla.csv', index=True)